In [48]:
import numpy as np
import sys
sys.path.append(r"C:\Users\gapar\Logical-Clifford-Synthesis")
from Algorithms.algorithm_1 import SymplecticOperations
from helper_functions.helperfunctions import SymplecticMatrix
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Initialize the SymplecticOperations class
symplectic_ops = SymplecticOperations()
def gf2matinv(matrix):
    """
    Compute the inverse of a matrix in GF(2) using Gaussian elimination.
    """
    matrix = np.array(matrix, dtype=np.int8) % 2  # Ensure the matrix is in GF(2) and explicitly set to an integer type
    m = len(matrix)
    aug_matrix = np.concatenate((matrix, np.eye(m, dtype=np.int8)), axis=1)  # Explicit dtype
    for col in range(m):
        for row in range(col, m):
            if aug_matrix[row, col]:
                if row != col:
                    # Swap rows
                    aug_matrix[[col, row]] = aug_matrix[[row, col]]
                break
        else:  # This else belongs to the for, executed only if no break occurred
            raise ValueError("Matrix is singular over GF(2)")

        for i in range(m):
            if i != col and aug_matrix[i, col]:
                # Use explicit type conversion if necessary
                aug_matrix[i] = (aug_matrix[i] + aug_matrix[col]) % 2  # This replaces the ^= operation for clarity and compatibility
    return aug_matrix[:, m:]


def intersect(arr1, arr2):
    """
    Find the intersection between two arrays and return indices.
    This mimics MATLAB's intersect with index outputs.
    """
    # Get the sorted intersection and their indices in arr1 and arr2
    common_elements, ind_arr1, ind_arr2 = np.intersect1d(arr1, arr2, return_indices=True)
    return ind_arr1

def setdiff(arr1, arr2):
    """
    Mimics MATLAB's setdiff, returning elements in arr1 not in arr2.
    """
    return np.setdiff1d(arr1, arr2)

def find_all_symp_mat(U, V, I, J):
    """
    Function to find all symplectic matrices F that satisfy U([I,m+J],:) * F = V.
    Rows of U must form a symplectic basis for F_2^(2m), i.e., U must satisfy U*Omega*U' = Omega,
    where Omega = [0 I_m; I_m 0].
    Number of rows of V must be equal to (len(I) + len(J)).
    """
    m = U.shape[1] // 2
    #print("m",m)
    Omega = np.block([[np.zeros((m, m)), np.eye(m)], [np.eye(m), np.zeros((m, m))]])
    #print(Omega)
    if not np.all((U @ Omega @ U.T) % 2 == Omega):
        print('\nInvalid matrix U in function find_all_symp_mat\n')
        return [None]

    I = np.array(I).flatten()
    print("I")
    print(I)
    J = np.array(J).flatten()
    print("J")
    print(J)
    Ibar = np.setdiff1d(np.arange(1, m+1), I)
    print("ibar")
    print(Ibar)
    Jbar = np.setdiff1d(np.arange(1, m+1), J)
    print("jbar")
    print(Jbar)
    alpha = len(Ibar) + len(Jbar)
    print("alpha")
    print(alpha)
    tot = 2 ** (alpha * (alpha + 1) // 2)
    print("tot" ,tot)
    F_all = [None] * tot
    print("U[np.ix_(np.concatenate([I-1, m+J-1])")
    print(U[np.ix_(np.concatenate([I-1, m+J-1]), )])
    F0, *_ = symplectic_ops.find_symp_mat_transvecs(U[np.ix_(np.concatenate([I-1, m+J-1]), )], V)  # Adjusted indexing
    print("f0",F0)
    A = (U @ F0) % 2
    print("A")
    print(A)
    Ainv = gf2matinv(A)
    print("Ainv")
    print(Ainv)
    IbJb = np.union1d(Ibar, Jbar) # Adjusted indexing
    print("ibjb" , IbJb)
    Basis = A[np.ix_(np.concatenate([IbJb-1, m+IbJb-1]), )]
    print("basis")
    print(Basis)
    Subspace = (np.array([list(format(i, f'0{2*len(IbJb)}b')) for i in range(2**(2*len(IbJb)))]).astype(int) @ Basis) % 2
    print("subspace")
    print(Subspace)
    Basis_fixed_I = intersect(IbJb, I)    
    print("Basis_fixed_I")
    print(Basis_fixed_I)
    Basis_fixed_J = intersect(IbJb, J)
    print("Basis_fixed_J")
    print(Basis_fixed_J)
    Basis_fixed = Basis_fixed_I
    print("Basis_fixed")
    print(Basis_fixed)
    Basis_free = np.setdiff1d(np.arange(2*len(IbJb)), Basis_fixed)
    print("Basis_free")
    print(Basis_free)

    Choices = [None] * alpha
    print("choices")
    #print(Choices)

    for i in range(alpha):
        ind = Basis_free[i]
        print("ind")
        print(ind)
        h = np.zeros(len(Basis_fixed))
        print("h")
        print(h)
        if i < len(Ibar):
            print("len(Ibar)")
            print(len(Ibar))
            h[Basis_fixed == len(IbJb) + ind] = 1
        else:
            h[Basis_fixed == ind - len(IbJb)] = 1
        print("h")
        print(h)

        Innpdts = (Subspace @ np.fft.fftshift(Basis[Basis_fixed, :], axes=1).T) % 2
        Choices[i] = Subspace[np.all(np.mod(Innpdts, 2) == h, axis=1), :]

    for l in range(tot):
        Bl = A.copy()
        W = np.zeros((alpha, 2*m))
        lbin = format(l, f'0{alpha*(alpha+1)//2}b')
        v1_ind = int(lbin[:alpha], 2)
        W[0, :] = Choices[0][v1_ind, :]
        for i in range(1, alpha):
            vi_ind = int(lbin[sum(range(alpha, alpha-i, -1)):(sum(range(alpha, alpha-i, -1))+alpha-i)], 2)
            Innprods = (Choices[i] @ np.fft.fftshift(W, axes=1).T) % 2
            h = np.zeros(alpha)
            if i >= len(Ibar):
                h[Basis_free == Basis_free[i] - len(IbJb)] = 1
            Ch_i = Choices[i][np.all(np.mod(Innprods, 2) == h, axis=1), :]
            W[i, :] = Ch_i[vi_ind, :]
        Bl[np.ix_(np.concatenate([Ibar-1, m+Jbar-1]), )] = W  # Adjusted indexing
        F = (Ainv @ Bl) % 2
        F_all[l] = (F0 @ F) % 2

    return F_all


[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
X = np.array([[1, 0,0,0], [0, 1,0,0],[0,0,1,0],[0,0,0,1]])
Y = np.array([[1,1,0,0], [0, 1,0,0],[0,0,1,0],[0,0,1,1]])
I = [1,2]

J = [1]
F_all = find_all_symp_mat(X, Y, I, J)


[[0 1]
 [1 0]]
I
[1 2]
J
[1]
ibar
[]
jbar
[2]
alpha
1
tot 2
U[np.ix_(np.concatenate([I-1, m+J-1])
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]]
f0 [[1. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 1. 1.]]
A
[[1. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 1. 1.]]
Ainv
[[1 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 1 1]]
ibjb [2]
basis
[[0. 1. 0. 0.]
 [0. 1. 1. 1.]]
subspace
[[0. 0. 0. 0.]
 [0. 1. 1. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 1.]]
Basis_fixed_I
[0]
Basis_fixed_J
[]
Basis_fixed
[0]
Basis_free
[1]
choices
ind
1
h
[0.]
h
[1.]


In [50]:
print(F_all)

[array([[1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 1., 1.]]), array([[1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 1.]])]
